In [59]:
from dotenv import load_dotenv
load_dotenv()

from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [60]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
collection_name = "samsung2025"
vectorstore = Chroma(
    embedding_function= embeddings,
    persist_directory="../7_vectorstore/samsung_2025_db",
    collection_name=collection_name
)


In [61]:
ret_similarity = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":30}
)
ret_similarity.invoke("삼성전자")

[Document(id='7ef39294-b0e0-4bf5-8386-abb0aa455d04', metadata={'trapped': '/False', 'page': 4, 'page_label': '5', 'producer': 'Adobe PDF Library 15.0', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'creationdate': '2025-07-10T16:11:16+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2025-09-04T16:51:11+09:00', 'total_pages': 87}, page_content='삼성전자 지속가능경영보고서 2025\n05\nOur Company AppendixFacts & Figures PrinciplePlanet People\n회사소개\nAbout Us\n삼성전자주식회사(이하 삼성전자)는 인재와 기술을 기반으로 최고의 제품과 서비스를 창출함으로써 인류사회에 기여하는 글로벌 초일류 기업을 지향합니다. 이를 위해 삼성전자가 지켜나갈 약속인 5가지 경영원칙 을 세부원칙과 \n행동지침으로 구체화하고, 삼성전자 임직원이 지켜야 할 행동규범 으로 제정하여 모든 경영활동의 기준으로 삼고 있습니다. 앞으로도 삼성전자는 조직문화에 5가지 핵심가치 를 내재화하여 지속적으로 성장해갈 것입니다.\n사업부문 및 글로벌 네트워크 소개\n삼성전자는 제품 특성에 따라 DX(Device eXperience)와 DS(Device Solutions) 2개 부문으로 나뉘어 독립적으로 운영되고 있습니다. DX부문은 스마트폰, 네트워크 시스템, 컴퓨터, TV, 냉장고, 세탁기, 에어컨, 의료기기 등 완제품을 \n생산·판매하고 있으며, DS부문은 메모리 사업, Foundry 사업, System LSI 사업으로 구성되어 DRAM, NAND Flash, 모바일AP 등의 반도체 부품을 생산

In [62]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker


model_name = "cross-encoder/ms-marco-MiniLM-L6-v2"
max_length = 512
hf_ce = HuggingFaceCrossEncoder(
    model_name=model_name, model_kwargs={"device": "cuda", "max_length": max_length}
)

reranker = CrossEncoderReranker(
    model=hf_ce, top_n=10
)

In [63]:
reranker

CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=CrossEncoder(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 384, padding_idx=0)
        (position_embeddings): Embedding(512, 384)
        (token_type_embeddings): Embedding(2, 384)
        (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-5): 6 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=384, out_features=384, bias=True)
                (key): Linear(in_features=384, out_features=384, bias=True)
                (value): Linear(in_features=384, out_features=384, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
        

In [64]:
from langchain.retrievers import ContextualCompressionRetriever
com_retriever = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=ret_similarity
)

In [65]:
from langchain_community.document_transformers import LongContextReorder
reorder = LongContextReorder()

In [ ]:
def format_docs(docs):
    result = []
    for item in docs:
        result.append(item.page_content)
    
    return "\n\n--\n\n".join(result)    

In [67]:
from langchain_openai import ChatOpenAI
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 Context만 근거로 간결하고 정확하게 답하도록 해라.
     
     [Context]
     {context}

     """),
     ("human", "{question}"),
])
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
parser = StrOutputParser()

In [68]:

chain = rag_prompt | model | parser 
chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 Context만 근거로 간결하고 정확하게 답하도록 해라.\n\n     [Context]\n     {context}\n\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002911A4E5210>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029118409410>, root_client=<openai.OpenAI object at 0x00000291053A0490>, root_async_client=<openai.AsyncOpenAI object at 0x000002911A5DE8D0>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser()

# 1번 방법

In [ ]:
question = "삼성의 미래 계획은 어떻게 되나요?"
# response = rag_chain.invoke({"question":question})

# 2번 방법

In [82]:
com_retriever_chain = {
    "context": RunnableLambda(lambda x: x["question"])
    | com_retriever
    | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
}

In [83]:

rag_chain =  com_retriever_chain | chain

rag_chain.invoke({
    "question":"삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래 기술 리더십으로 재도약의 기반을 다지고, 새로운 영역에서 미래 성장동력을 확보해 나갈 계획입니다. 또한, 지속가능한 성장 기반 마련을 위해 이해관계자의 의견에 귀 기울이며 최선을 다할 예정입니다.'

# 3번 방법

In [ ]:
from operator import itemgetter

retriever_chain = com_retriever | RunnableLambda(lambda docs: reorder.transform_documents(docs)) | RunnableLambda(format_docs)

rag_chain = (
    RunnablePassthrough.assign(
        context = itemgetter("question") | retriever_chain
    )
    | rag_prompt
    | model 
    | parser 
)

rag_chain.invoke({
    "question":"삼성의 미래 계획은 어떻게 되나요?"
})

In [79]:
from langchain_openai import ChatOpenAI
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 Context만 근거로 간결하고 정확하게 답하도록 해라.
     
     [Context]
     {context}

     """),
     ("human", "{pro} 스타일에 맞게 {question}에 대답해라"),
])
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
parser = StrOutputParser()

In [ ]:

rag_chain =  (
    {
        "context": com_retriever_chain,
        "question" : RunnableLambda(lambda x : x['question']),
        "pro" : RunnableLambda(lambda x:x['pro'])
    } | chain
)
rag_chain.invoke({
    "question":"삼성의 미래 계획은 어떻게 되나요?",
    "pro":"냥냥체"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출해 인류사회에 공헌하며, 기술 리더십으로 재도약의 기반을 다지고 새로운 영역에서 미래 성장동력을 확보할 계획입니다. 또한, 지속가능경영을 강화하고 청소년 교육, 중소기업·스타트업 지원, 환경경영 등 다양한 분야에서 사회적 가치를 창출하며 지속가능한 성장 기반을 마련할 예정입니다.'